# Deep Learning for NLP - Project

RULES:

* Do not create any additional cell

* Fill in the blanks

* All cells should be runnable (modulo trivial compatibility bugs that we'd fix)

* 4 / 20 points will be allocated to the clarity of your code

* Efficient code will have a bonus

DELIVERABLE:

* this notebook
* the predictions of the SST test set

DO NOT INCLUDE THE DATASETS IN THE DELIVERABLE..

In [1]:
import io
import os
import numpy as np
import scipy

import pandas as pd
from urllib2 import urlopen

In [2]:
PATH_TO_DATA = "/Users/florian/Documents/ETUDES/Etudes Post-Prepa/4. ESSEC MSc DSBA/T2/6. Deep Learning/assignment_2/data/"

# 1) Monolingual (English) word embeddings 

In [3]:
class Word2vec():
    def __init__(self, fname, nmax=100000):
        if fname[0:4] == 'http':
            self.load_wordvec_url(fname, nmax)
        else:
            self.load_wordvec(fname, nmax)
        self.word2id = dict.fromkeys(self.word2vec.keys())
        self.id2word = {v: k for k, v in self.word2id.items()}
        self.embeddings = np.array(self.word2vec.values())
    
    def load_wordvec(self, fname, nmax):
        self.word2vec = {}
        with io.open(fname, encoding='utf-8') as f:
            next(f)
            for i, line in enumerate(f):
                word, vec = line.split(' ', 1)
                self.word2vec[word] = np.fromstring(vec, sep=' ')
                if i == (nmax - 1):
                    break
        print('Loaded %s pretrained word vectors from file' % (len(self.word2vec)))
    
    def load_wordvec_url(self, fname, nmax):
        url = fname
        self.word2vec={}
        f = urlopen(url) 
        for i, line in enumerate(f):
            if i == 0: 
                pass
            elif i > nmax:
                break
            else:
                word, vec = line.split(' ', 1)
                self.word2vec[word] = np.fromstring(vec, sep=' ')
        print('Loaded {} pretrained word vectors from URL {}'.format(nmax,url))

    def most_similar(self, w, K=5):
        # K most similar words: self.score  -  np.argsort 
        similar_dico={}
        for j in self.word2vec.keys():
            if j!=w:
                similar_dico[j] = self.score(w,j)
            else:
                pass
    
        return sorted(similar_dico, key = similar_dico.get, reverse = True)[0:K]

    def score(self, w1, w2):
        # cosine similarity: np.dot  -  np.linalg.norm
        v1, v2 = self.word2vec[w1], self.word2vec[w2]
        
        cos_sim = np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
        return cos_sim


In [4]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=200000)
print '\n'
# You will be evaluated on the output of the following:
for w1, w2 in zip(('cat', 'dog', 'dogs', 'paris', 'germany'), ('dog', 'pet', 'cats', 'france', 'berlin')):
    print(w1, w2, w2v.score(w1, w2))
print '\n'
for w1 in ['cat', 'dog', 'dogs', 'paris', 'germany']:
    print(w2v.most_similar(w1))

Loaded 200000 pretrained word vectors from file


('cat', 'dog', 0.67168366627924914)
('dog', 'pet', 0.68420640296692192)
('dogs', 'cats', 0.70743893280524039)
('paris', 'france', 0.77751085412885612)
('germany', 'berlin', 0.7420295235998392)


[u'cats', u'kitty', u'kitten', u'feline', u'kitties']
[u'dogs', u'puppy', u'Dog', u'doggie', u'canine']
[u'dog', u'pooches', u'Dogs', u'doggies', u'canines']
[u'france', u'Paris', u'parisian', u'london', u'berlin']
[u'austria', u'europe', u'german', u'berlin', u'poland']


In [5]:
class BoV():
    def __init__(self, w2v):
        self.w2v = w2v
    
    def encode(self, sentences, idf=False):
        # takes a list of sentences, outputs a numpy array of sentence embeddings
        # see TP1 for help
        sentemb = []
        for sent in sentences:
            if idf is False:
                # mean of word vectors
                sent_vec = [self.w2v.word2vec[w] for w in sent if w in self.w2v.word2vec]
            
            else:
                # idf-weighted mean of word vectors
                sent_vec = [self.w2v.word2vec[w]*idf[w] for w in sent 
                            if (w in self.w2v.word2vec) and (w in idf)]
            
            sentemb.append(np.mean(sent_vec, axis=0))
            
        if len(sentences) is 1:
            return np.reshape(sentemb, (300, ))
        return np.vstack(sentemb)

    def most_similar(self, s, sentences, idf=False, K=5):
        # get most similar sentences and **print** them
        keys = self.encode(sentences, idf)
        query = self.encode([s], idf)
        
        similar_dico={}
        sent_index = 0
        for sent_index, sent in enumerate(sentences):
            if sent != s:
                sv1 = query
                sv2 = keys[sent_index:sent_index+1][0]
                score = np.dot(sv1, sv2)/(np.linalg.norm(sv1)*np.linalg.norm(sv2))
                similar_dico[' '.join(sent)] = score
            else:
                pass
    
        result = sorted(similar_dico, key = similar_dico.get, reverse = True)[0:K]
        
        # Printing out.
        print("\nThe {} most similar sentences to '{}' are: ".format(K, ' '.join(s)))
        for i in range(K):
            print('{} - {}'.format(similar_dico[result[i]], result[i]))
        print('\n')
        return None
    
    def score(self, s1, s2, idf=False):
        # cosine similarity: use   np.dot  and  np.linalg.norm
        sv1, sv2 = self.encode([s1], idf), self.encode([s2], idf)
        
        score_sim = np.dot(sv1, sv2)/(np.linalg.norm(sv1)*np.linalg.norm(sv2))
        
        print("Sentences '{}' and '{}' have a similarity of {} \n".format(' '.join(s1), ' '.join(s2), score_sim))
        return None
    
    def build_idf(self, sentences):
        # build the idf dictionary: associate each word to its idf value
        idf = {}

        for sent in sentences:
            for w in set(sent):
                idf[w] = idf.get(w, 0) + 1
    
        idf = dict([(k,max(1, np.log10(len(sentences)/v))) for (k, v) in idf.items()])
        return idf

In [6]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=200000)
s2v = BoV(w2v)

# Load sentences in "PATH_TO_DATA/sentences.txt"
sentences = []
with open(PATH_TO_DATA+'/sentences.txt') as f:
    for i, line in enumerate(f):
        sent = line.rstrip().split()
        sentences.append(sent)

# Build idf scores for each word
idf = {} if True else s2v.build_idf(sentences)

# You will be evaluated on the output of the following:
print '\n-- BoV with the normal average of word vectors --'
s2v.most_similar('' if not sentences else sentences[10], sentences)  # BoV-mean
s2v.score('' if not sentences else sentences[7], '' if not sentences else sentences[13])

print '-- BoV with the weighted-average of word vectors - idf --'
idf = {} if False else s2v.build_idf(sentences) # with idf NOT empty
s2v.most_similar('' if not sentences else sentences[10], sentences, idf)  # BoV-idf
s2v.score('' if not sentences else sentences[7], '' if not sentences else sentences[13], idf)

Loaded 200000 pretrained word vectors from file
BoV with the normal average of word vectors

The 5 most similar sentences to '1 smiling african american boy .' are: 
0.917045335771 - an african american man smiling .
0.849855600388 - a little african american boy and girl looking up .
0.821713913954 - an afican american woman standing behind two small african american children .
0.820704738463 - an african american man is sitting .
0.819112249101 - a girl in black hat holding an african american baby .


Sentences '1 man singing and 1 man playing a saxophone in a concert .' and '10 people venture out to go crosscountry skiing .' have a similarity of 0.572625885972 

BoV with the weighted-average of word vectors - idf

The 5 most similar sentences to '1 smiling african american boy .' are: 
0.921780772743 - an african american man smiling .
0.871596000101 - an african american man is sitting .
0.865194017102 - a little african american boy and girl looking up .
0.85501990728 - an afican

# 2) Multilingual (English-French) word embeddings

Let's consider a bilingual dictionary of size V_a (e.g French-English).

Let's define **X** and **Y** the **French** and **English** matrices.

They contain the embeddings associated to the words in the bilingual dictionary.

We want to find a **mapping W** that will project the source word space (e.g French) to the target word space (e.g English).

Procrustes : **W\* = argmin || W.X - Y ||  s.t  W^T.W = Id**
has a closed form solution:
**W = U.V^T  where  U.Sig.V^T = SVD(Y.X^T)**

In what follows, you are asked to: 

In [9]:
# 1 - Download and load 50k first vectors of
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.vec

w2v_en = Word2vec('https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec', nmax=10000)
w2v_fr = Word2vec('https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.vec', nmax=10000)    


Loaded 10000 pretrained word vectors from URL https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec
Loaded 10000 pretrained word vectors from URL https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.vec


In [10]:
# 2 - Get words that appear in both vocabs (= identical character strings)
#     Use it to create the matrix X and Y (of aligned embeddings for these words)
#common_keys = list(set(eng_word_dico.keys()) & set(fr_word_dico.keys()))

x = [] #french
y = [] #english
for w in w2v_en.word2vec:
    if w in w2v_fr.word2vec:
        x.append(list(w2v_fr.word2vec[w]))
        y.append(list(w2v_en.word2vec[w]))
        
X = np.array(x).transpose() # X.shape = (300, nb)
Y = np.array(y).transpose() # Y.shape = (300, nb)

In [11]:
# 3 - Solve the Procrustes using the scipy package and: scipy.linalg.svd() and get the optimal W
#     Now W*French_vector is in the same space as English_vector


U, sigma, V_T = np.linalg.svd(np.dot(Y, X.transpose()))
W = np.dot(U, V_T)
W = np.mat(W) # W.shape = (300, 300)

In [12]:
def most_similar_wv(w2v, wv, K=5): #most similar words to an embedding
    top_list = []
    for word in w2v.word2vec:
        if score_wv(w2v.word2vec[word], wv) != 1 :
            top_list.append((word, score_wv(w2v.word2vec[word], wv)))
    return sorted(top_list, key=lambda x: x[1], reverse=True)[0:K]

def score_wv(wv1, wv2): # score between embeddings
    # cosine similarity: np.dot  -  np.linalg.norm
    return (np.dot(wv1, wv2) / (np.linalg.norm(wv1) * np.linalg.norm(wv2)))

In [13]:
# 4 - After alignment with W, give examples of English nearest neighbors of some French words (and vice versa)
#     You will be evaluated on that part and the code above


for w in ['cat', 'dog', 'germany']:
    print('\n'+'English nearest neighbors of: '+w)
    wv_y = w2v_en.word2vec[w]
    wv_x = list(np.array(np.dot(W.I, wv_y).transpose()).flatten())
    top_K_list = most_similar_wv(w2v_fr, wv_x)
    for x in top_K_list:
        print(x[0], x[1])
        
for w in ['chat', 'chien']:
    print('\n'+'French nearest neighbors of: '+w)
    wv_x = w2v_fr.word2vec[w]
    wv_y = list(np.array(np.dot(W, wv_x).transpose()).flatten())
    top_K_list = most_similar_wv(w2v_en, wv_y)
    for x in top_K_list:
        print(x[0], x[1])


English nearest neighbors of: cat
('cat', 0.69506881489576477)
('chat', 0.54310854040506906)
('lapin', 0.45907235725471146)
('dog', 0.45381997649092493)
('chien', 0.44317416357901185)

English nearest neighbors of: dog
('dog', 0.66211906797182563)
('chien', 0.61395931798880088)
('lapin', 0.51732327178586313)
('chiens', 0.51606329503889103)
('animal', 0.50373327231468534)

English nearest neighbors of: germany
('allemagne', 0.78598952926086108)
('berlin', 0.57407684315537366)
('allemande', 0.57177058697029637)
('autriche', 0.55911693677638608)
('weimar', 0.55678480364598815)

French nearest neighbors of: chat
('cat', 0.54310854040506917)
('rabbit', 0.50804231440890213)
('rat', 0.50393066237858286)
('dog', 0.48964155010460197)
('chat', 0.48144049187862209)

French nearest neighbors of: chien
('dog', 0.61395931798880132)
('dogs', 0.50530474132417136)
('rabbit', 0.44820124839291497)
('rat', 0.44433081064351448)
('cat', 0.44317416357901207)


If you want to dive deeper on this subject: https://github.com/facebookresearch/MUSE

# 3) Sentence classification with BoV and scikit-learn

In [14]:
# 1 - Load train/dev/test of Stanford Sentiment TreeBank (SST)
#     (https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)

sst_data = ['stsa.fine.train','stsa.fine.dev', 'stsa.fine.test.X', ]

def load_sst_dataset(f, label_known=True):
    x = []
    y = []
    with open(PATH_TO_DATA+'SST/'+f) as data:    
        for line in data:
            if label_known is True:
                label, sent = line.strip().split(' ', 1)
                x.append(sent)
                y.append(label)
            else:
                sent = line.strip()
                x.append(sent)
    return x,y if label_known else x

x_train, y_train = load_sst_dataset(sst_data[0])
x_dev, y_dev = load_sst_dataset(sst_data[1])
x_test = load_sst_dataset(sst_data[2],False)[0]

w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=200000)
s2v = BoV(w2v)

Loaded 200000 pretrained word vectors from file


In [15]:
# 2 - Encode sentences with the BoV model above

# we derive idf scores
idf = {} if False else s2v.build_idf(x_train + x_dev + x_test)# with idf NOT empty

# we create embeddings of sentences
x_train_bov = s2v.encode(x_train)
x_dev_bov = s2v.encode(x_dev)
x_test_bov = s2v.encode(x_test)

x_train_bov_idf = s2v.encode(x_train, idf)
x_dev_bov_idf = s2v.encode(x_dev, idf)
x_test_bov_idf = s2v.encode(x_test, idf)

/Users/florian/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if sys.path[0] == '':
/Users/florian/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [16]:
# 3 - Learn Logistic Regression on top of sentence embeddings using scikit-learn
#     (consider tuning the L2 regularization on the dev set)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print '- LogisticRegression -'
print '-- BoV with the normal average of word vectors --'
log_clf = LogisticRegression(solver='lbfgs', penalty='l2', max_iter=150, random_state=9, multi_class='multinomial')
log_clf.fit(x_train_bov, y_train)
preds = log_clf.predict(x_train_bov)
print('Accuracy on train set: {}'.format(accuracy_score(y_train, preds)))
preds = log_clf.predict(x_dev_bov)
print('Accuracy on dev set: {} \n'.format(accuracy_score(y_dev, preds)))

print '-- BoV with the weighted-average of word vectors - idf --'
log_clf = LogisticRegression(solver='lbfgs', penalty='l2', max_iter=150, random_state=9, multi_class='multinomial')
log_clf.fit(x_train_bov_idf, y_train)
preds = log_clf.predict(x_train_bov_idf)
print('Accuracy on train set: {}'.format(accuracy_score(y_train, preds)))
preds = log_clf.predict(x_dev_bov_idf)
print('Accuracy on dev set: {}'.format(accuracy_score(y_dev, preds)))

- LogisticRegression -
-- BoV with the normal average of word vectors --
Accuracy on train set: 0.306062734082
Accuracy on dev set: 0.319709355132 

-- BoV with the weighted-average of word vectors - idf --
Accuracy on train set: 0.309691011236
Accuracy on dev set: 0.320617620345


In [17]:
# 4 - Produce 2210 predictions for the test set (in the same order). One line = one prediction (=0,1,2,3,4).
#     Attach the output file "logreg_bov_y_test_sst.txt" to your deliverable.
#     You will be evaluated on the results of the test set.

y_test_preds = log_clf.predict(x_test_bov_idf)
df=pd.DataFrame(y_test_preds)
df.to_csv(PATH_TO_DATA+'/logreg_bov_y_test_sst.txt', header=None, index=None, sep=' ', mode='a')

In [18]:
# BONUS!
# 5 - Try to improve performance with another classifier
#     Attach the output file "XXX_bov_y_test_sst.txt" to your deliverable (where XXX = the name of the classifier)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# TYPE CODE HERE
print '- GradientBoostingClassifier -'
print '-- BoV with the normal average of word vectors --'
clf = GradientBoostingClassifier(n_estimators=20)
clf.fit(x_train_bov, y_train)
preds = clf.predict(x_train_bov)
print('Accuracy on train set: {}'.format(accuracy_score(y_train, preds)))
preds = clf.predict(x_dev_bov)
print('Accuracy on dev set: {} \n'.format(accuracy_score(y_dev, preds)))

print '-- BoV with the weighted-average of word vectors - idf --'
clf = GradientBoostingClassifier(n_estimators=20)
clf.fit(x_train_bov_idf, y_train)
preds = clf.predict(x_train_bov_idf)
print('Accuracy on train set: {}'.format(accuracy_score(y_train, preds)))
preds = clf.predict(x_dev_bov_idf)
print('Accuracy on dev set: {}'.format(accuracy_score(y_dev, preds)))

y_test_preds = clf.predict(x_test_bov)
df=pd.DataFrame(y_test_preds)
df.to_csv(PATH_TO_DATA+'/gradboost_bov_y_test_sst.txt', header=None, index=None, sep=' ', mode='a')

- GradientBoostingClassifier -
-- BoV with the normal average of word vectors --
Accuracy on train set: 0.393609550562
Accuracy on dev set: 0.300635785649 

-- BoV with the weighted-average of word vectors - idf --
Accuracy on train set: 0.397471910112
Accuracy on dev set: 0.297910990009


# 4) Sentence classification with LSTMs in Keras

## 4.1 - Preprocessing

In [19]:
import keras
from keras.preprocessing import sequence

Using TensorFlow backend.


In [20]:
# 1 - Load train/dev/test sets of SST

sst_data = ['stsa.fine.train','stsa.fine.dev', 'stsa.fine.test.X', ]

def load_sst_dataset(f, label_known=True):
    x = []
    y = []
    with open(PATH_TO_DATA+'SST/'+f) as data:    
        for line in data:
            if label_known is True:
                label, sent = line.strip().split(' ', 1)
                x.append(sent)
                y.append(label)
            else:
                sent = line.strip()
                x.append(sent)
    return x,y if label_known else x

x_train, y_train = load_sst_dataset(sst_data[0])
x_dev, y_dev = load_sst_dataset(sst_data[1])
x_test = load_sst_dataset(sst_data[2],False)[0]

In [21]:
# 2 - Transform text to integers using keras.preprocessing.text.one_hot function
#     https://keras.io/preprocessing/text/

from keras.preprocessing import text
N = 50000
max_len = 60

x_train_oh = [text.one_hot(i, n=N, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n', lower=True, split=' ') for i in x_train]
x_dev_oh = [text.one_hot(i, n=N, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n', lower=True, split=' ') for i in x_dev]
x_test_oh = [text.one_hot(i, n=N, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n', lower=True, split=' ') for i in x_test]

**Padding input data**

Models in Keras (and elsewhere) take batches of sentences of the same length as input. It is because Deep Learning framework have been designed to handle well Tensors, which are particularly suited for fast computation on the GPU.

Since sentences have different sizes, we "pad" them. That is, we add dummy "padding" tokens so that they all have the same length.

The input to a Keras model thus has this size : (batchsize, maxseqlen) where maxseqlen is the maximum length of a sentence in the batch.

In [22]:
# 3 - Pad your sequences using keras.preprocessing.sequence.pad_sequences
#     https://keras.io/preprocessing/sequence/

x_train = keras.preprocessing.sequence.pad_sequences(x_train_oh, maxlen=max_len, dtype='int32', padding='pre', truncating='pre', value=0.)
x_dev = keras.preprocessing.sequence.pad_sequences(x_dev_oh, maxlen=max_len, dtype='int32', padding='pre', truncating='pre', value=0.)
x_test = keras.preprocessing.sequence.pad_sequences(x_test_oh, maxlen=max_len, dtype='int32', padding='pre', truncating='pre', value=0.)

from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_dev = to_categorical(y_dev)


## 4.2 - Design and train your model

In [23]:
# 4 - Design your encoder + classifier using keras.layers
#     In Keras, Torch and other deep learning framework, we create a "container" which is the Sequential() module.
#     Then we add components to this contained : the lookuptable, the LSTM, the classifier etc.
#     All of these components are contained in the Sequential() and are trained together.


# ADAPT CODE BELOW


from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation

embed_dim  = 32  # word embedding dimension
nhid       = 64  # number of hidden units in the LSTM
vocab_size = N # size of the vocabulary
n_classes  = 5

model = Sequential()
model.add(Embedding(vocab_size, embed_dim))
model.add(LSTM(nhid, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(n_classes, activation='sigmoid'))


/Users/florian/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, dropout=0.2, recurrent_dropout=0.2)`


In [24]:
# 5 - Define your loss/optimizer/metrics

# MODIFY CODE BELOW

loss_classif     =  'categorical_crossentropy' # find the right loss for multi-class classification
optimizer        =  'RMSprop' # find the right optimizer
metrics_classif  =  ['accuracy']

# Observe how easy (but blackboxed) this is in Keras
model.compile(loss=loss_classif,
              optimizer=optimizer,
              metrics=metrics_classif)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          1600000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 1,625,157
Trainable params: 1,625,157
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
# 6 - Train your model and find the best hyperparameters for your dev set
#     you will be evaluated on the quality of your predictions on the test set

# ADAPT CODE BELOW
bs = 32
n_epochs = 3

history = model.fit(x_train, y_train, batch_size=bs, 
                    nb_epoch=n_epochs, validation_data=(x_dev, y_dev))

/Users/florian/anaconda2/lib/python2.7/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 8544 samples, validate on 1101 samples
Epoch 1/3
8544/8544 [==============================] - 63s 7ms/step - loss: 1.5717 - acc: 0.2687 - val_loss: 1.5666 - val_acc: 0.3206
Epoch 2/3
8544/8544 [==============================] - 65s 8ms/step - loss: 1.5074 - acc: 0.3329 - val_loss: 1.4648 - val_acc: 0.3497
Epoch 3/3
8544/8544 [==============================] - 58s 7ms/step - loss: 1.3659 - acc: 0.4012 - val_loss: 1.3931 - val_acc: 0.3833


In [26]:
# 7 - Generate your predictions on the test set using model.predict(x_test)
#     https://keras.io/models/model/
#     Log your predictions in a file (one line = one integer: 0,1,2,3,4)
#     Attach the output file "logreg_lstm_y_test_sst.txt" to your deliverable.

y_pred = model.predict_classes(x_test)
with open(PATH_TO_DATA+"logreg_lstm_y_test_sst.txt", "w") as f:
    for i in y_pred:
        f.write(str(i)+'\n')

2210/2210 [==============================] - 2s 894us/step


## 4.3 -- innovate !

In [ ]:
# 8 - Open question: find a model that is better on your dev set
#     (e.g: use a 1D ConvNet, use a better classifier, pretrain your lookup tables ..)
#     you will get point if the results on the test set are better: be careful of not overfitting your dev set too much..
#     Attach the output file "XXX_XXX_y_test_sst.txt" to your deliverable.

# TYPE CODE HERE
